Hey there!

I used this codebook to organise data from multiple excel sheets into one, and to translate the texts from Danish to English (or the other way around) using DeepL API.

_**Created by:** Csilla Duray_

# 1. Importing libraries

## 1.1 Installing packages

In [1]:
!pip install translate --quiet

In [2]:
!pip install deepl --quiet

## 1.2 Adding source and path to find files (Google Colab)

In [ ]:
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
path = ''       # add your path here 
os.chdir(path)

Mounted at /content/drive












# 2. Loading data

## 2.1 Libraries and functions

In [4]:
import pandas as pd
import numpy as np
import re

In [ ]:
# translation
from translate import Translator
import deepl
translator = deepl.Translator('') # add your token here

In [ ]:
def replace(text, replacements=None, lower=True):
    '''Replaces multiple characters defined in a dictionary in textual data.'''
    if lower == True:
      text = text.lower()

    if replacements != None:
      for old, new in replacements.items():
          text = str(text).replace(old, new)

    return text

## 2.2 Data loading and preparation

In [ ]:
# importing the excel with all sheets - takes 30 sec
dfs_import = pd.read_excel('ndhvndata_clean_200923_MMnamedtabs.xlsx', sheet_name=None)

In [ ]:
# renaming sheets for easier coding
replacements = {'ø': 'o', '+': '', '-': '_', ' ': '_', '__': '_'}

new_keys = [replace(key, replacements) for key in dfs_import.keys()]
dfs = dict(zip(new_keys, dfs_import.values()))

In [ ]:
dfs.keys()

dict_keys(['respondents', 'home', 'sheet_names_code', 'important_places_obro', 'disliked_places_obro', 'important_places_ndhvn', 'wild_nature', 'lawns_flowers', 'social_areas', 'sports_facilities', 'facilities_other', 'other', 'no_wild_nature', 'no_lawns_flowers', 'no_social_areas', 'no_sports_facilities', 'no_facilities_other', 'no_other'])

In [ ]:
# defining my sheets of interest
sheets = {'important_places_obro': 'Important Places - Østerbro',
          'disliked_places_obro': 'Disliked Places - Østerbro',
          'important_places_ndhvn': 'Important Places - Nordhavn'}

In [ ]:
dfs['important_places_obro'].columns

Index(['Respondent ID', 'Publication ID', 'Submitted', 'Submitted Time',
       'First Active', 'Last Active', 'Publication Consent',
       'Participatory Consent', 'Approved By Moderator', 'Hidden By Moderator',
       'Identified', 'Processed', 'Language', 'Index', 'Zoom', 'wkt',
       'geojson', 'OB_UGS', 'OB_UGS_Reason', 'OB_UGS_Frequency',
       'OB_UGS_Relax', 'OB_UGS_Natural', 'OB_UGS_Aesthetic', 'OB_UGS_Food',
       'OB_UGS_Social', 'OB_UGS_Identitiy', 'OB_UGS_Community',
       'OB_UGS_Learning', 'OB_UGS_Climate', 'OB_UGS_Wilderness',
       'OB_UGS_Dogwalking', 'OB_UGS_Swimming', 'OB_UGS_Sports',
       'OB_UGS_Boating', 'OB_UGS_Walking', 'OB_UGS_Outdoor',
       'OB_UGS_Other_specified'],
      dtype='object')

In [ ]:
dfs['important_places_ndhvn'].head()
# strip beginning of col and merge

,respondent_id,language,wkt,ugs_reason,ugs_frequency,ugs_relax,ugs_natural,ugs_aesthetic,ugs_food,ugs_social,...,ugs_learning,ugs_climate,ugs_wilderness,ugs_dogwalking,ugs_swimming,ugs_sports,ugs_boating,ugs_walking,ugs_outdoor,ugs_other_specified
0,7zfc37wx7lo4,da,POINT (12.595124 55.706328),Bringer os tæt på havet,15.0,1,1,1,1,1,...,1,1,1,1,0,1,1,0,1,NaN
1,7aaj8kg67m36,da,POINT (12.584507 55.720214),Sol og vand,25.0,0,1,1,1,1,...,1,1,1,1,0,1,1,1,1,NaN
2,3zn6v4bl3jv3,da,POINT (12.595391 55.706108),Svømning,10.0,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,NaN
3,9ec9tvx3ktz9,da,POINT (12.604939 55.708185),Da denne badebro stadig var tilgængelig brugte...,0.0,0,1,1,1,0,...,1,1,1,1,0,1,1,1,1,NaN
4,9ec9tvx3ktz9,da,POINT (12.594263 55.71756),Jeg nyder af og til en sommeraften her.,5.0,0,1,1,1,0,...,1,1,1,1,1,1,1,1,1,NaN


In [ ]:
dfs['disliked_places_obro'].head()
# merge
# remove first row

,respondent_id,language,wkt,ugs_reason,ugs_frequency
4,9ec9tvx3ktz9,da,POINT (12.604852 55.708186),Før i tiden lå der en skøn badebro her. Nu er ...,0
5,6nk9zg7vo2c6,da,POINT (12.567325 55.718956),Meget grint byggeri,2
6,9na36ho8d3na,da,POINT (12.564342 55.695065),Amorparken er for støjplaget til at være rigti...,2
7,3xn9vag7b8p3,da,POINT (12.568769 55.692822),Det er bare en kedelig græsplæne ud til en ret...,1
8,3xn9vag7b8p3,da,POINT (12.560723 55.712977),Det er en ret tom plads med meget lidt grønt. ...,15


In [ ]:
for key in sheets.keys():
  # rename columns
  dfs[key].rename(columns=lambda col: replace(col, {' ': '_', 'ob_':'', 'nh_':''}), inplace=True)

  # keep specific columns
  cols = ['respondent_id', 'language', 'wkt'] + [col for col in dfs[key].columns if col.startswith('ugs_')]
  dfs[key] = dfs[key][cols]

In [ ]:
# dislikes: removing the first (empty) row and renaming cols
rename_cols = ['']

for key in ['disliked_places_obro']:
  # removing first indexing row
  dfs[key] = dfs[key].drop(dfs[key].index[0])

  # removing '_dislike' from column names
  dfs[key].rename(columns={col: col.replace('_dislike', '') for col in dfs[key].columns}, inplace=True)

  # drop ugs
  del dfs[key]['ugs']

In [ ]:
# bringing it all into one dataframe
important_places_df = pd.DataFrame()

for key in sheets.keys():
  df = dfs[key].copy()

  if 'disliked' in key:
    df['dislike'] = 1
  else:
    df['dislike'] = 0
  df['area'] = key.split('_')[-1]

  important_places_df = pd.concat([important_places_df, df], ignore_index=True)

In [ ]:
len(dfs['important_places_ndhvn']) + len(dfs['important_places_obro']) + len(dfs['disliked_places_obro']) == len(important_places_df)

True

In [ ]:
# dislike = 1 if dislike
important_places_df.head()

,respondent_id,language,wkt,ugs_reason,ugs_frequency,ugs_relax,ugs_natural,ugs_aesthetic,ugs_food,ugs_social,...,ugs_wilderness,ugs_dogwalking,ugs_swimming,ugs_sports,ugs_boating,ugs_walking,ugs_outdoor,ugs_other_specified,dislike,area
0,7zfc37wx7lo4,da,POINT (12.59515 55.706415),Det bringer os tæt på vandet,15.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,NaN,0,obro
1,7zfc37wx7lo4,da,POINT (12.570357 55.699256),Vi går og løber mange ture i Fælledparken og g...,30.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,Frirum til børn,0,obro
2,37xba4xw9db3,da,POINT (12.579697 55.685548),,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0,obro
3,37xba4xw9db3,da,POINT (12.580035 55.686086),,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0,obro
4,3bko6m7wi6w6,da,POINT (12.553122 55.716613),Jeg går ture og leger dernede med niecer og ne...,200.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,0,obro


In [ ]:
# replace nan with ''
important_places_df['ugs_reason'].fillna('', inplace=True)

## 2.3 Translation - EN to DA

In [ ]:
def change_language(text, input_lang='en', target_lang='da'):
    '''Translates text from English to Danish'''
    translated_text = translator.translate_text(text.lower(), target_lang=target_lang)
    return translated_text

In [ ]:
# LAST RUNTIME: 04/10/2024 08:09
# translate and saving translated df to save on translations - takes ca. 4 min, ca. 46 273 characters
df = important_places_df.copy()

for col in ['ugs_reason']:
  condition = (df['language'] == 'en') | (df['language'] == 'en, da')
  df.loc[condition, col] = df.loc[condition, col].apply(lambda x: change_language(str(x)) if x != '' else x)

df.to_csv('important_places_translated.csv', index = False)

In [ ]:
# checking output
df[df.language=='en, da'].head()

,respondent_id,language,wkt,ugs_reason,ugs_frequency,ugs_relax,ugs_natural,ugs_aesthetic,ugs_food,ugs_social,...,ugs_wilderness,ugs_dogwalking,ugs_swimming,ugs_sports,ugs_boating,ugs_walking,ugs_outdoor,ugs_other_specified,dislike,area
449,2iw7eut66x3a,"en, da",POINT (12.553755 55.713588),,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0,obro
450,2iw7eut66x3a,"en, da",POINT (12.527691 55.719561),hvor jeg går,30.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,0,obro
511,2p44kk6bbr27,"en, da",POINT (12.584482 55.719066),Det er beroligende at se på havet,100.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,Running,0,obro
678,39xjc73gdl24,"en, da",POINT (12.57071 55.698972),,NaN,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,0,obro
994,8sf44e2ez628,"en, da",POINT (12.569976 55.699761),,10.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,0,obro


## 2.3 Translation - DA to EN
- keeping the code, didn't run this time

In [ ]:
# def change_language(text, input_lang='da', target_lang='en-gb'):
#     '''Translates text from Danish to English'''
#     translated_text = translator.translate_text(text, target_lang=target_lang)
#     return translated_text

In [ ]:
# # LAST RUNTIME: 10/05/2024 12:26
# # translate and saving translated df to save on translations - took ca. 21 min, ca. 302 489 characters
# df = wishes_df.copy()

# for col in ['description', 'reason']:
#   condition = (df['language'] == 'da') | (df['language'] == 'da, en')
#   df.loc[condition, col] = df.loc[condition, col].apply(lambda x: change_language(str(x)) if x != '' else x)

In [ ]:
# # preparing the final wishes dataset
# df['description'] = df['description'].replace('Dyr', 'Animals')
# df['comment'] = df['description']+' '+df['reason']
# df = df[['respondent_id', 'wkt', 'category', 'wish', 'comment']]
# df.head(2)

In [ ]:
# df.to_csv('wishes_translated_en.csv', index=False)

# Next steps are in the codebooks:
**Output data:** 'important_places_translated.csv'

**Previously created:** 'participants_translated.csv'

**Next notebook:** 2-important-places-content-analysis-04-10-2024